In [173]:
#Import libraries
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score as ss
import pandas as pd 
import numpy as np
import plotly.express as px
import itertools as it 
import folium

In [174]:
#Original dataframe
original_df = pd.read_excel('Street Data_BaDinh.xlsx')
#Drop duplicates
new_df = original_df.drop_duplicates(subset=['Longitude','Latitude'])
new_df.head(10)



,Index,District,Ward,Street Name 1,Street name 2,Street Name 3,Street Name 4,Longitude,Latitude,Traffic Level
0,10101,Ba Đình,Phúc Xá,An Xá,Nghĩa Dũng,NaN,NaN,21.046224,105.849289,1
1,10102,Ba Đình,Phúc Xá,An Xá,Tân Mỹ,NaN,NaN,21.042412,105.852127,1
2,10103,Ba Đình,Phúc Xá,An Xá,Phúc Xá,NaN,NaN,21.046067,105.849022,1
3,11901,Ba Đình,Điện Biên,Bà Huyện Thanh Quan,Chùa Một Cột,NaN,NaN,21.034974,105.835874,1
4,11902,Ba Đình,Điện Biên,Bà Huyện Thanh Quan,Lê Hồng Phong,NaN,NaN,21.033849,105.835607,2
5,11903,Ba Đình,Điện Biên,Bắc Sơn,Hoàng Diệu,NaN,NaN,21.035962,105.839265,1
6,11904,Ba Đình,Điện Biên,Bắc Sơn,Tôn Thất Đàm,NaN,NaN,21.035980,105.838074,1
7,11905,Ba Đình,Điện Biên,Bắc Sơn,Độc Lập,NaN,NaN,21.036582,105.836578,1
8,11601,Ba Đình,Ngọc Hà,Bắc Sơn,Ngọc Hà,NaN,NaN,21.037553,105.831080,1
9,10701,Ba Đình,Cống Vị,Bưởi,Đội Cấn,NaN,NaN,21.037688,105.807348,3


In [175]:
#Count Missing Value
print(new_df.isnull().sum())
#Check Datatype
new_df.dtypes

Index              0
District           0
Ward               0
Street Name 1      0
Street name 2      0
Street Name 3    165
Street Name 4    194
Longitude          0
Latitude           0
Traffic Level      0
dtype: int64


Index              int64
District          object
Ward              object
Street Name 1     object
Street name 2     object
Street Name 3     object
Street Name 4     object
Longitude        float64
Latitude         float64
Traffic Level      int64
dtype: object

In [176]:
fig = px.bar(new_df, x="Ward", color=new_df["Traffic Level"].astype(str), color_discrete_map={"3": "red","1": "green","2": "yellow"})
fig.show()

In [177]:
#Count
CountsTotal = []
Ward_Count_Total = new_df['Ward'].value_counts()
for i in range (0, len(Ward_Count_Total)): 
    CountsTotal.append(Ward_Count_Total[i])
px.bar(Ward_Count_Total, y = CountsTotal)

In [178]:
#Cordinates and Traffic Level
main = new_df.iloc[:,[7,8,9]]
main.head(10)

,Longitude,Latitude,Traffic Level
0,21.046224,105.849289,1
1,21.042412,105.852127,1
2,21.046067,105.849022,1
3,21.034974,105.835874,1
4,21.033849,105.835607,2
5,21.035962,105.839265,1
6,21.035980,105.838074,1
7,21.036582,105.836578,1
8,21.037553,105.831080,1
9,21.037688,105.807348,3


In [179]:
#Cordinates for Traffic Level 1 
TF_1 = main[main['Traffic Level'] == 1].iloc[:,[0,1]]
TF_1.head(10)

,Longitude,Latitude
0,21.046224,105.849289
1,21.042412,105.852127
2,21.046067,105.849022
3,21.034974,105.835874
5,21.035962,105.839265
6,21.035980,105.838074
7,21.036582,105.836578
8,21.037553,105.831080
20,21.042534,105.841838
21,21.043551,105.842065


In [180]:
#Cordinates for Traffic Level 2
TF_2 = main[main['Traffic Level'] == 2].iloc[:,[0,1]]
TF_2.head(10)

,Longitude,Latitude
4,21.033849,105.835607
10,21.036802,105.806949
14,21.030388,105.838591
15,21.029578,105.841557
16,21.030178,105.836997
18,21.028600,105.805231
19,21.028430,105.805509
25,21.034975,105.836843
26,21.033652,105.836641
33,21.040651,105.842084


In [181]:
#Cordinates for Traffic Level 3
TF_3 = main[main['Traffic Level'] == 3].iloc[:,[0,1]]
TF_3

,Longitude,Latitude
9,21.037688,105.807348
11,21.034335,105.805245
12,21.029744,105.802636
13,21.047508,105.806128
27,21.031582,105.836236
28,21.030390,105.836007
38,21.045600,105.843339
44,21.032382,105.809576
48,21.030593,105.815841
49,21.023419,105.819744


In [182]:
#Defining Models and Clusters\
cluster1 = TF_1.to_numpy()
cluster2 = TF_2.to_numpy()
cluster3 = TF_3.to_numpy()

#Training Model for Traffic Level 1
model1 = DBSCAN(0.005,min_samples=5).fit(cluster1)
TF_1['labels'] = model1.labels_
print((TF_1['labels']).value_counts())
score1 = ss(cluster1, TF_1['labels'])
score1

after_fig1 = px.scatter(TF_1, 
                        x = "Latitude",
                        y = "Longitude", 
                        color=TF_1['labels'].astype(str),
                        labels={"color" : "Group"},
                      ) #Default eps, min
after_fig1.update_layout(
            title={
            'text' : "Area with traffic level 1",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig1

#Training Model for Traffic Level 2
model2 = DBSCAN(0.005, min_samples=5).fit(cluster2)
TF_2['labels'] = model2.labels_
print(TF_2['labels'].value_counts())
score2 = ss(cluster2, TF_2['labels'])
score2

after_fig2 = px.scatter(TF_2,
                        x = "Latitude" , 
                        y = "Longitude", 
                        color=TF_2['labels'].astype(str),
                        )
after_fig2.update_layout(
            title={
            'text' : "Area with traffic level 2",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig2

#Training Model for Traffic Level 3
model3 = DBSCAN(0.005,min_samples=5).fit(cluster3) 
TF_3['labels']= model3.labels_
print(TF_3['labels'].value_counts())
score3 = ss(cluster3, TF_3['labels'])
score3

after_fig3 = px.scatter(TF_3, 
                        x = "Latitude",
                        y = "Longitude", 
                        color=TF_3['labels'].astype(str))
after_fig3.update_layout(
            title={
            'text' : "Area with traffic level 3",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig3

In [183]:
#Range of Parameters
epsilon1 = np.linspace(0.0003, 0.0049, num=1000).tolist()
epsilon2 = np.linspace(0.0013, 0.0073, num=1000).tolist()
epsilon3 = np.linspace(0.001, 0.0098, num=1000).tolist()
min_samples = [3,4,5,6,7]
combinations1 = list(it.product(epsilon1, min_samples))
combinations2 = list(it.product(epsilon2, min_samples))
combinations3 = list(it.product(epsilon3, min_samples))
len(combinations1 + combinations2 + combinations3)

15000

In [184]:
#Case 2: Using mupitle functions to get the best score
def getScore1(combinations_x):
    loop_count = 1
    #Loop through parameters from combinations
    for i, (epsilon, min_samples) in enumerate(combinations_x):
        #Fit model
        model1 = DBSCAN(eps = epsilon, min_samples = min_samples).fit(cluster1)
        #Get lables
        TF_1['labels'] = model1.labels_
        num_clus = len(TF_1['labels'].value_counts()) 
        #Get scores
        if num_clus > 2:
            score = ss(cluster1, TF_1['labels'])
            if score > 0.41 and num_clus>3: 
                print("Loop number:",loop_count,"| Parameters:", epsilon,",", min_samples,"| Score for Trafic Level 1:", score,"| Number of clusters:", num_clus)
        loop_count += 1
def getScore2(combinations_x):
    loop_count = 1
    #Loop through parameters from combinations
    for i, (epsilon, min_samples) in enumerate(combinations_x):
        #Fit model
        model2 = DBSCAN(eps = epsilon, min_samples = min_samples).fit(cluster2)
        #Get lables
        TF_2['labels'] = model2.labels_
        num_clus = len(TF_2['labels'].value_counts()) 
        #Get scores
        if num_clus > 2:
            score = ss(cluster2, TF_2['labels']) 
            if score > 0.4569 and num_clus > 3:
                print("Loop number:",loop_count,"| Parameters:", epsilon,",", min_samples,"| Score for Trafic Level 2:", score,"| Number of clusters:", num_clus)
        loop_count += 1
def getScore3(combinations_x):
    loop_count = 1
    #Loop through parameters from combinations
    for i, (epsilon, min_samples) in enumerate(combinations_x):
        #Fit model
        model3 = DBSCAN(eps = epsilon, min_samples = min_samples).fit(cluster3)
        #Get lables
        TF_3['labels'] = model3.labels_
        num_clus = len(TF_3['labels'].value_counts()) 
        #Get scores
        if num_clus > 2:
            score = ss(cluster3, TF_3['labels']) 
            if score > 0.517 and num_clus > 3:
                print("Loop number:",loop_count,"| Parameters:", epsilon,",", min_samples,"| Score for Trafic Level 3:", score,"| Number of clusters:", num_clus)
        loop_count += 1

In [185]:
getScore1(combinations1)

Loop number: 2890 | Parameters: 0.0029568568568568566 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2895 | Parameters: 0.0029614614614614614 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2900 | Parameters: 0.002966066066066066 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2905 | Parameters: 0.0029706706706706704 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2910 | Parameters: 0.002975275275275275 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2915 | Parameters: 0.00297987987987988 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2920 | Parameters: 0.0029844844844844846 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2925 | Parameters: 0.002989089089089089 , 7 | Score for Trafic Level 1: 0.4148858

In [186]:
getScore2(combinations2)

Loop number: 2917 | Parameters: 0.0048015015015015015 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 2922 | Parameters: 0.004807507507507507 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 2927 | Parameters: 0.004813513513513513 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 2932 | Parameters: 0.004819519519519519 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 2937 | Parameters: 0.004825525525525526 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 2942 | Parameters: 0.0048315315315315315 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 2947 | Parameters: 0.004837537537537537 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4


In [187]:
getScore3(combinations3)

Loop number: 2726 | Parameters: 0.005800800800800801 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2731 | Parameters: 0.00580960960960961 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2736 | Parameters: 0.005818418418418418 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2741 | Parameters: 0.0058272272272272265 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2746 | Parameters: 0.0058360360360360354 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2751 | Parameters: 0.005844844844844844 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2756 | Parameters: 0.005853653653653653 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 2761 | Parameters: 0.005862462462462462 , 3 | Score for Trafic Level 3: 0.5171859719938379

TF1
Loop number: 2317 | Parameters: 0.002963963963963964 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2325 | Parameters: 0.0029738738738738736 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4
Loop number: 2333 | Parameters: 0.002983783783783784 , 7 | Score for Trafic Level 1: 0.41488584456394306 | Number of clusters: 4

TF2
Loop number: 3802 | Parameters: 0.004807207207207207 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 3810 | Parameters: 0.004817117117117117 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 3818 | Parameters: 0.0048270270270270275 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
Loop number: 3826 | Parameters: 0.004836936936936937 , 4 | Score for Trafic Level 2: 0.4569707840575685 | Number of clusters: 4
<-------------------------------------------------------------------------------------------------------------------------------->
Loop number: 3801 | Parameters: 0.004807207207207207 , 3 | Score for Trafic Level 2: 0.4567671412096184 | Number of clusters: 5
Loop number: 3809 | Parameters: 0.004817117117117117 , 3 | Score for Trafic Level 2: 0.4567671412096184 | Number of clusters: 5
Loop number: 3817 | Parameters: 0.0048270270270270275 , 3 | Score for Trafic Level 2: 0.4567671412096184 | Number of clusters: 5
Loop number: 3825 | Parameters: 0.004836936936936937 , 3 | Score for Trafic Level 2: 0.4567671412096184 | Number of clusters: 5

TF3
Loop number: 4601 | Parameters: 0.005798198198198198 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4609 | Parameters: 0.005808108108108109 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4617 | Parameters: 0.005818018018018018 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4625 | Parameters: 0.005827927927927928 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4633 | Parameters: 0.005837837837837838 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4641 | Parameters: 0.005847747747747748 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4649 | Parameters: 0.005857657657657658 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4657 | Parameters: 0.005867567567567568 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4665 | Parameters: 0.005877477477477478 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4673 | Parameters: 0.005887387387387388 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4681 | Parameters: 0.005897297297297297 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4
Loop number: 4689 | Parameters: 0.005907207207207207 , 3 | Score for Trafic Level 3: 0.5171859719938379 | Number of clusters: 4

In [188]:
#Training Model for Traffic Level 1
model1 = DBSCAN(0.00296,min_samples=7).fit(cluster1)
TF_1['labels'] = model1.labels_
print((TF_1['labels']).value_counts())
score1 = ss(cluster1, TF_1['labels'])
score1

 1    28
 0    15
-1    10
 2     6
Name: labels, dtype: int64


0.41488584456394306

In [189]:
after_fig1 = px.scatter(TF_1, 
                        x = "Latitude",
                        y = "Longitude", 
                        color=TF_1['labels'].astype(str),
                        color_discrete_map={'-1':'red', '0': '#636EFA'},
                        labels={"color" : "Group"},
                      ) #Default eps, min
after_fig1.update_layout(
            title={
            'text' : "Area with traffic level 1 with eps = 0.00296, min_samples = 7",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig1

In [190]:
#Training Model for Traffic Level 2
model2 = DBSCAN(0.00481, min_samples=4).fit(cluster2)
TF_2['labels'] = model2.labels_
print(TF_2['labels'].value_counts())
score2 = ss(cluster2, TF_2['labels'])
score2

 0    62
 1    14
-1     8
 2     7
Name: labels, dtype: int64


0.4569707840575685

In [191]:
after_fig2 = px.scatter(TF_2,
                        x = "Latitude" , 
                        y = "Longitude", 
                        color=TF_2['labels'].astype(str),
                        )
after_fig2.update_layout(
            title={
            'text' : "Area with traffic level 2 with eps = 0.00481, min_samples = 4",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig2

In [192]:
#Training Model for Traffic Level 3
model3 = DBSCAN(0.00581,min_samples=3).fit(cluster3) 
TF_3['labels']= model3.labels_
print(TF_3['labels'].value_counts())
score3 = ss(cluster3, TF_3['labels'])
score3

 0    28
 1    10
 2     5
-1     4
Name: labels, dtype: int64


0.5171859719938379

In [193]:
after_fig3 = px.scatter(TF_3,
                        x = "Latitude" , 
                        y = "Longitude", 
                        color=TF_3['labels'].astype(str),
                        )
after_fig3.update_layout(
            title={
            'text' : "Area with traffic level 3 with eps = 0.0058, min_samples = 3",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig3

In [239]:
#Sample 1
def createMap(dataset1, dataset2, dataset3):
    m = folium.Map(location=[21.034388, 105.831716], zoom_start=14, tiles='cartodbpositron')
    group_tf_1 = folium.FeatureGroup("Traffic Level 1").add_to(m)
    group_tf_2 = folium.FeatureGroup("Traffic Level 2").add_to(m)
    group_tf_3 = folium.FeatureGroup("Traffic Level 3").add_to(m)
    labs1 = dataset1['labels'].to_numpy()
    labs2 = dataset2['labels'].to_numpy()
    labs3 = dataset3['labels'].to_numpy()
    long_lat1 = dataset1.iloc[:,[0,1]].to_numpy()
    long_lat2 = dataset2.iloc[:,[0,1]].to_numpy()
    long_lat3 = dataset3.iloc[:,[0,1]].to_numpy()
    for i in range (0, len(labs1)):
        if labs1[i] == 0:
            folium.CircleMarker(long_lat1[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == 1:
            folium.CircleMarker(long_lat1[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == 2: 
            folium.CircleMarker(long_lat1[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == -1: 
            folium.CircleMarker(long_lat1[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_1)
    for i in range (0, len(labs2)):
        if labs2[i] == 0:
            folium.CircleMarker(long_lat2[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == 1:
            folium.CircleMarker(long_lat2[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == 2: 
            folium.CircleMarker(long_lat2[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == -1: 
            folium.CircleMarker(long_lat2[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_2)
    for i in range (0, len(labs3)):
        if labs3[i] == 0:
            folium.CircleMarker(long_lat3[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == 1:
            folium.CircleMarker(long_lat3[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == 2: 
            folium.CircleMarker(long_lat3[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == -1: 
            folium.CircleMarker(long_lat3[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_3)
    m.add_child(folium.LayerControl(collapsed=False)) 
    return m


In [240]:
createMap1(TF_1, TF_2,TF_3)

In [244]:
def createMap2(dataset1, dataset2, dataset3):
    m = folium.Map(location=[21.034388, 105.831716], zoom_start=14, tiles='cartodbpositron')
    group_tf_1 = folium.FeatureGroup("Traffic Level 1").add_to(m)
    group_tf_2 = folium.FeatureGroup("Traffic Level 2").add_to(m)
    group_tf_3 = folium.FeatureGroup("Traffic Level 3").add_to(m)
    labs1 = dataset1['labels'].to_numpy()
    labs2 = dataset2['labels'].to_numpy()
    labs3 = dataset3['labels'].to_numpy()
    long_lat1 = dataset1.iloc[:,[0,1]].to_numpy()
    long_lat2 = dataset2.iloc[:,[0,1]].to_numpy()
    long_lat3 = dataset3.iloc[:,[0,1]].to_numpy()
    for i in range (0, len(labs1)):
        if labs1[i] == 0:
            folium.CircleMarker(long_lat1[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == 1:
            folium.CircleMarker(long_lat1[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == 2: 
            folium.CircleMarker(long_lat1[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == -1: 
            folium.CircleMarker(long_lat1[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_1)
    for i in range (0, len(labs2)):
        if labs2[i] == 0:
            folium.CircleMarker(long_lat2[i], radius= 1, color="#FFA15A",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == 1:
            folium.CircleMarker(long_lat2[i], radius= 1, color="#19D3F3",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == 2: 
            folium.CircleMarker(long_lat2[i], radius= 1, color="#FF6692",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == -1: 
            folium.CircleMarker(long_lat2[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_2)
    for i in range (0, len(labs3)):
        if labs3[i] == 0:
            folium.CircleMarker(long_lat3[i], radius= 1, color="#2CA02C",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == 1:
            folium.CircleMarker(long_lat3[i], radius= 1, color="#FF97FF",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == 2: 
            folium.CircleMarker(long_lat3[i], radius= 1, color="#FECB52",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == -1: 
            folium.CircleMarker(long_lat3[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_3)
    m.add_child(folium.LayerControl(collapsed=False)) 
    return m


In [245]:
createMap2(TF_1, TF_2,TF_3)